# Terra boilerplate and smart contracts handlers

In [ ]:
#============================ Imports ============================#

from terra_sdk.client.localterra import LocalTerra, LCDClient
from terra_sdk.util.contract import read_file_as_b64, get_code_id
from terra_sdk.core.wasm import MsgStoreCode
from terra_sdk.core.auth import StdFee
from terra_sdk.core.wasm import MsgInstantiateContract
from terra_sdk.util.contract import get_contract_address
from terra_sdk.core.wasm import MsgExecuteContract
from terra_sdk.core.coins import Coins
from terra_sdk.core.bank import MsgSend
from terra_sdk.core.coins import Coins
from terra_sdk.client.localterra import Wallet
import base64
import json
import chalk

#============================ Get Terra and accounts ============================#

terra = LocalTerra()

# Builtin wallets from LocalTerra
deployer = terra.wallets["test1"]
alice = terra.wallets["test2"]
bob = terra.wallets["test3"]
mallory = terra.wallets["test4"]

#============================ SDK wrappers ============================#

def store_contract(terra: LCDClient, sender: Wallet, contract_name: str) -> str:
    """Uploads contract, returns code ID"""
    contract_bytes = read_file_as_b64(f"../artifacts/{contract_name}.wasm")
    # Don't forget `.key.acc_address``
    store_code = MsgStoreCode(
        sender=sender.key.acc_address, wasm_byte_code=contract_bytes)
    tx = sender.create_and_sign_tx(
        msgs=[store_code], fee=StdFee(10_000_000, "10000000uluna"))
    result = terra.tx.broadcast(tx)
    try:
        code_id = get_code_id(result)
        print(chalk.green(f"[+] Code ID of {contract_name}: {code_id}"))
    except ValueError as e:
        print(chalk.red(f"[!] Error storing contract {contract_name}"))
        print(result)
        raise e
    return code_id


def instantiate_contract(terra: LCDClient, sender: Wallet, contract_id: str, init_msg: dict) -> str:
    """Instantiate contract, returns the contract address"""
    # Admin must be deployer or tx returns Unauthorized
    instantiate = MsgInstantiateContract(
        sender=sender.key.acc_address, admin=sender.key.acc_address, code_id=contract_id, init_msg=init_msg)
    tx = sender.create_and_sign_tx(
        msgs=[instantiate], fee=StdFee(10_000_000, "10000000uluna"))
    result = terra.tx.broadcast(tx)
    try:
        contract_address = get_contract_address(result)
        print(chalk.green(
            f"[+] Contract ID {contract_id} is instantiated at: {contract_address}"))
    except ValueError as e:
        print(chalk.red(f"[!] Error instantiating contract ID {contract_id}"))
        print(result)
        raise e
    return contract_address


def execute_contract(terra: LCDClient, sender: Wallet, contract_address: str, execute_msg: dict, init_coins: Coins = None) -> str:
    """Execute a message"""
    execute = MsgExecuteContract(sender=sender.key.acc_address,
                                 contract=contract_address, execute_msg=execute_msg, coins=init_coins)
    tx = sender.create_and_sign_tx(
        msgs=[execute], fee=StdFee(10_000_000, "10000000uluna"))
    result = terra.tx.broadcast(tx)
    return result


def send(terra: LCDClient, sender: Wallet, to_address: str, amount=None) -> str:
    """Send coins"""
    send_msg = MsgSend(from_address=sender.key.acc_address,
                       to_address=to_address, amount=amount)
    tx = sender.create_and_sign_tx(msgs=[send_msg], fee=StdFee(
        1000000, "1000000uusd"), fee_denoms=['uusd', 'uluna', 'ukrw'])
    result = terra.tx.broadcast(tx)
    return result


def to_binary(o: dict):
    return base64.b64encode(json.dumps(o).encode()).decode()


# Contract abstraction and messages declaration

In [ ]:
class Contract:
    def __init__(self, name: str="contract") -> None:
        self.name = name

    def query(self, query_msg):
        query_res = terra.wasm.contract_query(self.address, query_msg)
        return query_res
    
    def execute(self, sender: Wallet, execute_msg):
        execute_result = execute_contract(terra, sender, self.address, execute_msg)
        return execute_result
    
    def instantiate(self, sender: Wallet, contract_id: str, init_msg):
        self.address = instantiate_contract(terra, sender, contract_id, init_msg)
        return self.address

class CW20(Contract):
    def __init__(self, name: str="cw20") -> None:
        self.name = name

    def instantiate_msg(name: str, symbol: str, decimals: int, initial_balances = [], mint = None):
        return {
            "decimals": decimals,
            "name": name,
            "initial_balances": initial_balances,
            "symbol": symbol,
            "mint": mint,
        }

    def query_balance_msg(address: str):
        return {"balance": {"address": address}}

    def query_token_info_msg():
        return {"token_info": {}}

    def query_minter_msg():
        return {"minter": {}}

    def query_allowance_msg(owner: str, spender: str):
        return {"allowance": {"owner": owner, "spender": spender}}

    def query_all_allowance_msg(owner: str, start_after=None, limit=None):
        return {"all_allowances": {"owner": owner, "start_after": start_after, "limit": limit}}

    def query_all_accounts_msg(start_after=None, limit=None):
        return {"all_accounts": {"start_after": start_after, "limit": limit}}
    
    def execute_transfer_msg(recipient: str, amount: str):
        return {"transfer": {"recipient": recipient, "amount": amount}}

    def execute_burn_msg(amount: str):
        return {"burn": {"amount": amount}}

    def execute_send_msg(contract: str, amount: str, msg=None):
        return {"send": {"contract": contract, "amount": amount, "msg": msg}}
    
    def execute_mint_msg(recipient: str, amount: str):
        return {"mint": {"recipient": recipient, "amount": amount}}
    
    def execute_increase_allowance_msg(spender: str, amount: str, expires=None):
        return {"increase_allowance": {"spender": spender, "amount": amount, "expires": expires}}
        
    def execute_decrease_allowance_msg(spender: str, amount: str, expires=None):
        return {"decrease_allowance": {"spender": spender, "amount": amount, "expires": expires}}

    def execute_transfer_from_msg(owner: str, recipient: str, amount: str):
        return {"transfer_from": {"owner": owner, "recipient": recipient, "amount": amount}}

    def execute_send_from_msg(owner: str, contract: str, amount: str, msg=None):
        return {"send_from": {"owner": owner, "contract": contract, "amount": amount, "msg": msg}}

    def execute_burn_from_msg(owner: str, amount: str):
        return {"burn_from": {"owner": owner, "amount": amount}}

class AnchorToken(CW20):
    def __init__(self, name: str="AnchorToken") -> None:
        self.name = name

    def instantiate_msg(name: str, symbol: str, decimals: int, reward_contract: str, initial_balances = [], mint = None):
        return {
            "decimals": decimals,
            "name": name,
            "initial_balances": initial_balances,
            "reward_contract": reward_contract,
            "symbol": symbol,
            "mint": mint,
        }

class AnchorReward(Contract):
    def __init__(self, name: str="AnchorReward") -> None:
        self.name = name

    def instantiate_msg(owner: str, reward_denom: str):
        return {"owner": owner, "reward_denom": reward_denom}

    def query_config_msg():
        return {"config": {}}

    def query_state_msg():
        return {"state": {}}

    def query_accrued_rewards_msg(address: str):
        return {"accrued_rewards": {"address": address}}

    def query_holder_msg(address: str):
        return {"holder": {"address": address}}

    def query_holders_msg(start_after=None, limit=None):
        return {"holders": {"start_after": start_after, "limit": limit}}

    def execute_post_initialize_msg(token_contract: str):
        return {"post_initialize": {"token_contract": token_contract}}

    def execute_update_config_msg(owner: str):
        return {"update_config": {"owner": owner}}

    def execute_increase_balance_msg(address: str, amount: str):
        return {"increase_balance": {"address": address, "amount": amount}}

    def execute_decrease_balance_msg(address: str, amount: str):
        return {"decrease_balance": {"address": address, "amount": amount}}

    def execute_claim_rewards_msg(recipient: str = None):
        return {"claim_rewards": {"recipient": recipient}}

class AnchorConverter(Contract):  
    def __init__(self, name: str="AnchorConverter") -> None:
        self.name = name
  
    def instantiate_msg(owner: str):
        return {"owner": owner}

    def execute_register_tokens(wormhole_token_address: str, anchor_token_address: str):
        return {"register_tokens": {"wormhole_token_address": wormhole_token_address, "anchor_token_address": anchor_token_address}}

    def receive_convert_wormhole_to_anchor():
        return {"convert_wormhole_to_anchor": {}}

    def receive_convert_anchor_to_wormhole():
        return {"convert_anchor_to_wormhole": {}}

class WormholeToken(CW20):
    def __init__(self, name: str="WormholeToken") -> None:
        self.name = name

# Instantiation and interaction

In [ ]:
#============================ Store contracts ============================#

wormhole_token_code_id = store_contract(terra, deployer, 'terraswap_token')
anchor_reward_code_id = store_contract(terra, deployer, 'anchor_bsol_reward')
anchor_token_code_id = store_contract(terra, deployer, 'anchor_bsol_token')
anchor_converter_code_id = store_contract(terra, deployer, 'anchor_bsol_converter')

#======================== Create contract classes ========================#

wormhole_token = WormholeToken("WormholeToken")
anchor_reward = AnchorReward("AnchorReward")
anchor_token = AnchorToken("AnchorToken")
anchor_converter = AnchorConverter("AnchorConverter")

#============================ Init contracts ============================#

anchor_reward_init_msg = AnchorReward.instantiate_msg(deployer.key.acc_address, 'uusd')
res = anchor_reward.instantiate(deployer, anchor_reward_code_id, anchor_reward_init_msg)
anchor_token_init_msg = AnchorToken.instantiate_msg('bsol', 'BSOL', 6, anchor_reward.address, [], {'minter': deployer.key.acc_address})
res = anchor_token.instantiate(deployer, anchor_token_code_id, anchor_token_init_msg)
res = anchor_reward.execute(deployer, AnchorReward.execute_post_initialize_msg(anchor_token.address))
